<a href="https://colab.research.google.com/github/kavish-24/Konkani_Mental_Health/blob/main/AudioHandling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install openai-whisper pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 17.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=108f0f12024b107c96f04060774553fa464f2087de92ec8b7fb2eebfebd181df
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 21.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=b762e624668be70d6834a57a55083c2a7874c72c01aa845f2d42daa25ca6be90
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [2]:
import whisper
from pydub import AudioSegment
import os

def split_audio_on_word_boundaries(audio_path, max_duration=30, output_dir="segments"):
    """
    Split audio into segments of max_duration seconds, ending at word boundaries.

    Args:
        audio_path: Path to input audio file
        max_duration: Maximum segment duration in seconds (default: 30)
        output_dir: Directory to save output segments
    """

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Load Whisper model (use 'base' for speed, 'small'/'medium' for accuracy)
    print("Loading Whisper model...")
    model = whisper.load_model("base")

    # Transcribe audio with word-level timestamps
    print(f"Transcribing {audio_path}...")
    result = model.transcribe(audio_path, word_timestamps=True)

    # Load audio file
    print("Loading audio file...")
    audio = AudioSegment.from_file(audio_path)

    # Extract all words with timestamps
    words = []
    for segment in result['segments']:
        if 'words' in segment:
            words.extend(segment['words'])

    if not words:
        print("No words detected in audio!")
        return

    # Split into segments
    segments = []
    current_start = 0
    max_duration_ms = max_duration * 1000

    for i, word in enumerate(words):
        word_end_ms = word['end'] * 1000

        # Check if adding this word would exceed max duration
        if word_end_ms - (current_start * 1000) >= max_duration_ms:
            # End segment at previous word boundary
            if i > 0:
                segment_end = words[i-1]['end']
                segments.append((current_start, segment_end))
                current_start = segment_end
            else:
                # First word itself exceeds duration, include it anyway
                segment_end = word['end']
                segments.append((current_start, segment_end))
                current_start = segment_end

    # Add final segment
    if current_start < words[-1]['end']:
        segments.append((current_start, words[-1]['end']))

    # Export segments
    print(f"\nCreating {len(segments)} segments...")
    base_name = os.path.splitext(os.path.basename(audio_path))[0]

    for idx, (start, end) in enumerate(segments, 1):
        start_ms = int(start * 1000)
        end_ms = int(end * 1000)

        segment_audio = audio[start_ms:end_ms]
        output_path = os.path.join(output_dir, f"{base_name}_segment_{idx:03d}.mp3")

        segment_audio.export(output_path, format="mp3")
        duration = (end - start)
        print(f"Segment {idx}: {duration:.2f}s ({start:.2f}s - {end:.2f}s) -> {output_path}")

    print(f"\nDone! Created {len(segments)} segments in '{output_dir}' directory")
    return segments


def split_audio_aggressive(audio_path, max_duration=30, output_dir="segments"):
    """
    Alternative approach: More aggressive splitting for very fast speech.
    Uses silence detection as fallback if word boundaries are too far apart.
    """
    from pydub.silence import detect_nonsilent

    os.makedirs(output_dir, exist_ok=True)

    print("Loading Whisper model...")
    model = whisper.load_model("base")

    print(f"Transcribing {audio_path}...")
    result = model.transcribe(audio_path, word_timestamps=True)

    print("Loading audio file...")
    audio = AudioSegment.from_file(audio_path)

    # Extract words with timestamps
    words = []
    for segment in result['segments']:
        if 'words' in segment:
            words.extend(segment['words'])

    if not words:
        print("No words detected! Falling back to silence detection...")
        return split_on_silence_only(audio_path, max_duration, output_dir)

    # Detect silence periods (as backup split points)
    nonsilent_ranges = detect_nonsilent(audio, min_silence_len=100, silence_thresh=-40)

    segments = []
    current_start = 0
    max_duration_ms = max_duration * 1000

    for i, word in enumerate(words):
        word_end_ms = word['end'] * 1000
        current_duration = word_end_ms - (current_start * 1000)

        if current_duration >= max_duration_ms:
            # Try to find a word boundary close to max_duration
            target_time = current_start * 1000 + max_duration_ms

            # Find nearest word end before target time
            best_split = None
            for j in range(i, -1, -1):
                candidate_end = words[j]['end'] * 1000
                if candidate_end <= target_time:
                    best_split = words[j]['end']
                    break

            if best_split:
                segments.append((current_start, best_split))
                current_start = best_split
            else:
                # Use current word as split point
                segments.append((current_start, word['end']))
                current_start = word['end']

    # Add final segment
    if current_start < words[-1]['end']:
        segments.append((current_start, words[-1]['end']))

    # Export segments
    print(f"\nCreating {len(segments)} segments...")
    base_name = os.path.splitext(os.path.basename(audio_path))[0]

    for idx, (start, end) in enumerate(segments, 1):
        start_ms = int(start * 1000)
        end_ms = int(end * 1000)

        segment_audio = audio[start_ms:end_ms]
        output_path = os.path.join(output_dir, f"{base_name}_segment_{idx:03d}.mp3")

        segment_audio.export(output_path, format="mp3")
        duration = (end - start)
        print(f"Segment {idx}: {duration:.2f}s -> {output_path}")

    print(f"\nDone! Created {len(segments)} segments")
    return segments

def process_audio_directory(input_dir, output_base_dir, max_duration=30):
    audio_exts = (".wav", ".mp3", ".m4a", ".flac")

    print("Loading Whisper model once...")
    model = whisper.load_model("base")

    for file in os.listdir(input_dir):
        if not file.lower().endswith(audio_exts):
            continue

        audio_path = os.path.join(input_dir, file)
        audio_name = os.path.splitext(file)[0]

        # Create unique folder for each audio
        audio_output_dir = os.path.join(output_base_dir, audio_name)
        os.makedirs(audio_output_dir, exist_ok=True)

        print(f"\nProcessing {file}...")
        split_audio_on_word_boundaries(
            audio_path,
            max_duration=max_duration,
            output_dir=audio_output_dir
        )

# Example usage
if __name__ == "__main__":
    input_audio_folder = r"/content/drive/MyDrive/November 2017"
    output_segments_folder = r"/content/drive/MyDrive/Audio_segments/November"
    max_segment_duration = 25

    process_audio_directory(
        input_audio_folder,
        output_segments_folder,
        max_duration=max_segment_duration
    )

    print(f"Input audio: {audio_file}")
    print(f"Output directory: {output_directory}")
    print(f"Max segment duration: {max_segment_duration} seconds\n")

    # Basic splitting (recommended)
    segments = split_audio_on_word_boundaries(
        audio_file,
        max_duration=max_segment_duration,
        output_dir=output_directory
    )

    # Or use aggressive splitting for very fast speech
    # segments = split_audio_aggressive(
    #     audio_file,
    #     max_duration=max_segment_duration,
    #     output_dir=output_directory
    # )

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Loading Whisper model once...


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 56.8MiB/s]



Processing 06 nov 17_Konk Prime News.wav...
Loading Whisper model...
Transcribing /content/drive/MyDrive/November 2017/06 nov 17_Konk Prime News.wav...
Loading audio file...

Creating 17 segments...
Segment 1: 24.98s (0.00s - 24.98s) -> /content/drive/MyDrive/Audio_segments/November/06 nov 17_Konk Prime News/06 nov 17_Konk Prime News_segment_001.mp3
Segment 2: 24.76s (24.98s - 49.74s) -> /content/drive/MyDrive/Audio_segments/November/06 nov 17_Konk Prime News/06 nov 17_Konk Prime News_segment_002.mp3
Segment 3: 24.88s (49.74s - 74.62s) -> /content/drive/MyDrive/Audio_segments/November/06 nov 17_Konk Prime News/06 nov 17_Konk Prime News_segment_003.mp3
Segment 4: 24.68s (74.62s - 99.30s) -> /content/drive/MyDrive/Audio_segments/November/06 nov 17_Konk Prime News/06 nov 17_Konk Prime News_segment_004.mp3
Segment 5: 24.56s (99.30s - 123.86s) -> /content/drive/MyDrive/Audio_segments/November/06 nov 17_Konk Prime News/06 nov 17_Konk Prime News_segment_005.mp3
Segment 6: 23.96s (123.86s - 1

NameError: name 'audio_file' is not defined

In [ ]:
import os
import whisper
from pathlib import Path

def transcribe_mp3_directory(input_dir, output_dir=None, model_size="medium", language=None):
    """
    Transcribe all MP3 files in a directory using Whisper model.
    Skips files that already have a transcript in the output directory.
    """

    print(f"Loading Whisper {model_size} model...")
    model = whisper.load_model(model_size)

    input_path = Path(input_dir)
    output_path = Path(output_dir) if output_dir else input_path
    output_path.mkdir(parents=True, exist_ok=True)

    mp3_files = list(input_path.glob("*.mp3"))

    if not mp3_files:
        print(f"No MP3 files found in {input_dir}")
        return

    print(f"Found {len(mp3_files)} MP3 files to transcribe")
    print(f"Language setting: {'Auto-detect' if language is None else language}\n")

    successful = 0
    skipped = 0
    empty = 0

    for i, mp3_file in enumerate(mp3_files, 1):
        transcript_file = output_path / f"{mp3_file.stem}_transcript.txt"

        # ✅ Skip if file already exists
        if transcript_file.exists():
            print(f"[{i}/{len(mp3_files)}] Skipping {mp3_file.name} — transcript already exists.")
            skipped += 1
            continue

        print(f"[{i}/{len(mp3_files)}] Transcribing: {mp3_file.name}")

        try:
            transcribe_options = {
                "task": "transcribe",
                "fp16": False,
                "temperature": 0.0,
                "best_of": 5,
                "beam_size": 5,
            }

            if language:
                transcribe_options["language"] = language

            result = model.transcribe(str(mp3_file), **transcribe_options)
            transcript_text = result["text"].strip()

            if not transcript_text:
                print("⚠ Empty transcript (possibly silent audio)")
                empty += 1
            else:
                successful += 1
                print(f"✓ Detected language: {result.get('language', 'unknown')}")
                print(f"  Text preview: {transcript_text[:100]}{'...' if len(transcript_text) > 100 else ''}")

            with open(transcript_file, "w", encoding="utf-8") as f:
                f.write(transcript_text)

            print(f"  Saved to: {transcript_file.name}\n")

        except Exception as e:
            print(f"✗ Error transcribing {mp3_file.name}: {str(e)}\n")
            continue

    print("="*60)
    print("Transcription complete!")
    print(f"Successful: {successful} | Empty: {empty} | Skipped: {skipped} | Total: {len(mp3_files)}")
    print("="*60)


def transcribe_segment_directories(input_base_dir, output_base_dir, model_size="medium", language=None):
    """
    Transcribe MP3 segments inside each audio directory and save transcripts
    to matching output directories. Skips already transcribed files.
    """

    print(f"Loading Whisper {model_size} model...")
    model = whisper.load_model(model_size)

    input_base = Path(input_base_dir)
    output_base = Path(output_base_dir)
    output_base.mkdir(parents=True, exist_ok=True)

    for folder in sorted(input_base.iterdir()):
        if not folder.is_dir():
            continue

        print(f"\n📂 Processing folder: {folder.name}")
        out_folder = output_base / folder.name
        out_folder.mkdir(exist_ok=True)

        mp3_files = list(folder.glob("*.mp3"))
        if not mp3_files:
            print(f"  ⚠ No MP3 segments in {folder.name}")
            continue

        print(f"  Found {len(mp3_files)} segments")

        successful, skipped = 0, 0

        for i, mp3_file in enumerate(mp3_files, 1):
            output_txt = out_folder / f"{mp3_file.stem}.txt"

            # ✅ Skip if already exists
            if output_txt.exists():
                print(f"    [{i}/{len(mp3_files)}] Skipping {mp3_file.name} — transcript already exists.")
                skipped += 1
                continue

            print(f"    [{i}/{len(mp3_files)}] Transcribing {mp3_file.name}")

            try:
                options = {
                    "task": "transcribe",
                    "fp16": False,
                    "temperature": 0.0,
                    "best_of": 5,
                    "beam_size": 5,
                }

                if language:

                    options["language"] = language

                result = model.transcribe(str(mp3_file), **options)
                text = result["text"].strip()

                with open(output_txt, "w", encoding="utf-8") as f:
                    f.write(text)

                successful += 1

            except Exception as e:
                print(f"     ❌ Error: {e}")
                continue

        print(f"✅ Finished {folder.name} | Transcribed: {successful}, Skipped: {skipped}")


if __name__ == "__main__":
    input_segments = "/content/drive/MyDrive/Audio_segments/November"
    output_transcripts = "/content/drive/MyDrive/Segment transcript/November"

    transcribe_segment_directories(
        input_base_dir=input_segments,
        output_base_dir=output_transcripts,
        model_size="large-v2",
        language="mr"
    )


Loading Whisper large-v2 model...

📂 Processing folder: 03 nov 17_Konk Prime News
  Found 17 segments
    [1/17] Transcribing 03 nov 17_Konk Prime News_segment_001.mp3
    [2/17] Transcribing 03 nov 17_Konk Prime News_segment_002.mp3
    [3/17] Transcribing 03 nov 17_Konk Prime News_segment_003.mp3
    [4/17] Transcribing 03 nov 17_Konk Prime News_segment_004.mp3
    [5/17] Transcribing 03 nov 17_Konk Prime News_segment_005.mp3
    [6/17] Transcribing 03 nov 17_Konk Prime News_segment_006.mp3
    [7/17] Transcribing 03 nov 17_Konk Prime News_segment_007.mp3
    [8/17] Transcribing 03 nov 17_Konk Prime News_segment_008.mp3
    [9/17] Transcribing 03 nov 17_Konk Prime News_segment_009.mp3
    [10/17] Transcribing 03 nov 17_Konk Prime News_segment_010.mp3
    [11/17] Transcribing 03 nov 17_Konk Prime News_segment_011.mp3
    [12/17] Transcribing 03 nov 17_Konk Prime News_segment_012.mp3
    [13/17] Transcribing 03 nov 17_Konk Prime News_segment_013.mp3
    [14/17] Transcribing 03 nov 17_K